# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,teluknaga,-6.0989,106.6381,86.38,70,20,12.66,ID,1734435837
1,1,st. john's,47.5649,-52.7093,32.79,88,20,13.80,CA,1734435839
2,2,bonthe,7.5264,-12.5050,91.09,44,28,1.28,SL,1734435841
3,3,tadmur,34.5608,38.2805,54.37,36,6,3.80,SY,1734435844
4,4,lupane,-18.9315,27.8070,96.82,22,26,6.26,ZW,1734435846


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
#%%capture --no-display

# Configure the map plot
humidity_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    tiles = "OSM",
    frame_height = 504,
    frame_width = 896,
    hover_cols = ["City", "Humidity"]

    
)

# Display the map
humidity_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values 
ideal_city = city_data_df.loc[
    (city_data_df["Max Temp"] > 69.8) & (city_data_df["Max Temp"] < 80.6) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_city = ideal_city.dropna()

# Display sample data
print(len(ideal_city))
ideal_city.head()

3


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
128,128,hilo,19.7297,-155.0900,70.11,98,0,3.44,US,1734436109
144,144,neuquen,-38.9516,-68.0591,71.49,60,0,3.44,AR,1734436123
176,176,centenario,-38.8043,-68.1400,71.35,60,0,3.44,AR,1734435915


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
128,hilo,US,19.7297,-155.0900,98,
144,neuquen,AR,-38.9516,-68.0591,60,
176,centenario,AR,-38.8043,-68.1400,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accomodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hilo - nearest hotel: No hotel found
neuquen - nearest hotel: No hotel found
centenario - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
128,hilo,US,19.7297,-155.0900,98,No hotel found
144,neuquen,AR,-38.9516,-68.0591,60,No hotel found
176,centenario,AR,-38.8043,-68.1400,60,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display
hotel_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size = "Fahrenheit Temperature",
    scale = 1,
    color = "City Name",
    alpha = 0.5,
    tiles = "OSM",
    frame_height = 896,
    frame_width = 504,
    hover_cols = ["City Name", "Fahrenheit Temperature", "Hotel Name", "Hotel Address"]
)

hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]